In [1]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [3]:
%matplotlib

Using matplotlib backend: <object object at 0x000001D41A8F3A50>


In [7]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual Trial-And-Error Tuned Responses for Marble',fontsize = 15)
figraw.suptitle('Unfiltered Trial-And-Error Tuned Responses for Marble',fontsize = 15)
figz.suptitle('Motor Angle for Trial-And-Error Tuned Marble Response',fontsize = 15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Trial-And-Error Tuned Marble Responses',fontsize = 15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Marble Responses')
ax2.set_xlabel('Time [s]',fontsize = 15)
ax2.set_ylabel('Distance [cm]',fontsize = 15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(4, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1422
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('MARBLE_MAN_{}-Copy1.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]')
    axs.set_ylabel('Distance [cm]')
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize = 15)
    axsraw.set_ylabel('Distance [cm]',fontsize = 15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize = 15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize = 15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right')
axz.legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]



for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
        
Trial_Time = time_ave
Trial_Ave = filtered
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1554
1451
1460
1425
1422
[0.014000000000000002, 0.031, 0.049, 0.066, 0.084, 0.101, 0.118, 0.136, 0.1532, 0.172, 0.189, 0.207, 0.2242, 0.24139999999999998, 0.25939999999999996, 0.27640000000000003, 0.295, 0.3122, 0.3294, 0.3474, 0.36479999999999996, 0.38280000000000003, 0.4, 0.4178, 0.4364, 0.45380000000000004, 0.47359999999999997, 0.491, 0.5094000000000001, 0.5267999999999999, 0.5446000000000001, 0.5624, 0.5795999999999999, 0.5982000000000001, 0.6155999999999999, 0.6334, 0.651, 0.6686, 0.6866000000000001, 0.7038, 0.7224, 0.7398, 0.7576, 0.776, 0.7938000000000001, 0.8122, 0.8300000000000001, 0.849, 0.867, 0.885, 0.9044000000000001, 0.923, 0.942, 0.96, 0.9790000000000001, 0.9972, 1.0162, 1.0354, 1.054, 1.0734000000000001, 1.092, 1.1114000000000002, 1.1298, 1.1488, 1.1666, 1.1851999999999998, 1.2038, 1.2218, 1.2408000000000001, 1.2588, 1.2778, 1.2958, 1.3144000000000002, 1.333, 1.3512, 1.3708000000000002, 1.3887999999999998, 1.4083999999999999, 1.4265999999999999, 1.446, 1.464600000000000

In [8]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual Manually Tuned Responses for Ping Pong Ball',fontsize = 15)
figraw.suptitle('Unfiltered Manually Tuned Responses for Ping Pong Ball',fontsize = 15)
figz.suptitle('Motor Angle for Manually Tuned Ping Pong Ball Response',fontsize = 15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Manually Tuned Ping Pong Ball Responses',fontsize = 15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize = 15)
ax2.set_ylabel('Distance [cm]',fontsize = 15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1393
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('MARB_MAN_{}-Copy1.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]',fontsize = 15)
    axs.set_ylabel('Distance [cm]',fontsize = 15)
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize = 15)
    axsraw.set_ylabel('Distance [cm]',fontsize = 15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize = 15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize = 15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right')
axz.legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break

Manual_Time = time_ave
Manual_Ave = filtered
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1412
1393
2065
1422
2029
[0.014000000000000002, 0.0312, 0.0494, 0.0666, 0.08460000000000001, 0.10200000000000001, 0.11979999999999999, 0.137, 0.154, 0.17219999999999996, 0.18920000000000003, 0.20779999999999998, 0.22480000000000003, 0.24180000000000001, 0.2598, 0.277, 0.2954, 0.3126, 0.3298, 0.3478, 0.36500000000000005, 0.3834000000000001, 0.4006, 0.4184, 0.43579999999999997, 0.4534000000000001, 0.47140000000000004, 0.4888, 0.5072000000000001, 0.5246000000000001, 0.5426, 0.5614000000000001, 0.5798, 0.5993999999999999, 0.6177999999999999, 0.6374000000000001, 0.655, 0.6734000000000001, 0.6911999999999999, 0.7087999999999999, 0.7276, 0.7452, 0.7638, 0.7812000000000001, 0.799, 0.8173999999999999, 0.8351999999999998, 0.8539999999999999, 0.8712, 0.8902000000000001, 0.908, 0.9254000000000001, 0.9443999999999999, 0.9624, 0.9808, 0.9984, 1.0173999999999999, 1.0353999999999999, 1.0534, 1.0726, 1.0906000000000002, 1.1102, 1.1287999999999998, 1.1481999999999999, 1.1668, 1.1856, 1.2049999999999998,

In [9]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)
figraw,axsraw = plt.subplots()

figz,axz = plt.subplots(figsize = (15,5))
plt.subplots_adjust(right = 0.98,left = 0.05)

fig.suptitle('Individual SIMC Tuned Responses for Marble',fontsize = 15)
figraw.suptitle('Unfiltered SIMC Tuned Responses for Marble',fontsize = 15)
figz.suptitle('Motor Angle for SIMC Tuned Marble Response',fontsize = 15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of SIMC Tuned Marble Responses',fontsize = 15)
#fig3.suptitle('Unfiltered Average of Manually Tuned Marble Responses')
ax2.set_xlabel('Time [s]',fontsize = 15)
ax2.set_ylabel('Distance [cm]',fontsize = 15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 1000, 'lp', fs=50000, output='sos')
lines = ['solid','dashed','dotted','dashdot','solid']

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 1310
ticksang = np.linspace(-30,30,11)
for i in range(1,6):
    df = pd.read_csv('MARBLE_PID_{}-Copy1.csv'.format(i))
    
    time = ((df['time'])-5000)/1000
    print(len(time))
    timenew = time[0:END]
    #print(len(timenew))
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(timenew,filtered,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axs.set_xlabel('Time [s]',fontsize = 15)
    axs.set_ylabel('Distance [cm]',fontsize = 15)
    
    axs.grid()
    
    axsraw.plot(timenew,distance)
    axsraw.set_xlabel('Time [s]',fontsize = 15)
    axsraw.set_ylabel('Distance [cm]',fontsize = 15)
    axsraw.grid()
    
    axz.plot(timenew,angle_filt,label = 'Filtered {}'.format(i),linestyle = lines[i-1])
    axz.set_xlabel('Time [s]',fontsize = 15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize = 15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()

    
    
    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
axs.legend(loc = 'lower right')
axz.legend(loc = 'lower right')
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)
print(time_ave)
filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
for i in range(0,L-1):
    if time_ave[i]<1 and filtered[i]>filtered[i-1]:
        OS = filtered[i]
        OST = time_ave[i]
# OS = np.max(filtered)
# OSind = np.argmax(filtered)
# OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
        
        
SIMC_Time = time_ave
SIMC_Ave = filtered
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
#ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
#ax2.axhline(filtered[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid()
#ax2.plot(sett,Usett,'p',label = "settling point",color = 'g')
#ax2.vlines(time_ave[-1],ymin=14.95,ymax = filtered[-1]+0.05,color='r',label = "steady state error")
#ax2.plot(OST,OS,'x',color = 'r',label = 'overshoot')
ax2.legend(loc = 'lower right')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)

#ax3.plot(time_ave,dist_ave,label = 'response')

1479
1310
1639
1690
1705
[0.014000000000000002, 0.031, 0.0494, 0.0664, 0.0844, 0.1016, 0.119, 0.1368, 0.154, 0.1726, 0.1896, 0.20800000000000002, 0.225, 0.2426, 0.261, 0.2786, 0.2968, 0.3146, 0.33240000000000003, 0.35039999999999993, 0.368, 0.3864, 0.40359999999999996, 0.42160000000000003, 0.44000000000000006, 0.4574, 0.4757999999999999, 0.49379999999999996, 0.5119999999999999, 0.5298, 0.5474, 0.5656000000000001, 0.5831999999999999, 0.6012000000000001, 0.6188, 0.6368, 0.6548, 0.6721999999999999, 0.6904, 0.7078, 0.726, 0.7438, 0.7618, 0.78, 0.7978, 0.8163999999999998, 0.8337999999999999, 0.8523999999999999, 0.8705999999999999, 0.8882, 0.9065999999999999, 0.9246000000000001, 0.943, 0.9616, 0.9800000000000001, 0.9986, 1.0172, 1.0348000000000002, 1.0532, 1.0718, 1.0902, 1.1086, 1.127, 1.1454, 1.1634, 1.1818000000000002, 1.2004000000000001, 1.2186, 1.2372, 1.2558, 1.2746, 1.2924, 1.312, 1.3298, 1.3486, 1.3670000000000002, 1.3855999999999997, 1.4043999999999999, 1.4225999999999999, 1.4418000

In [11]:
fig7 = plt.figure()
plt.plot(Trial_Time,Trial_Ave,label='Trial-and-Error')
plt.plot(SIMC_Time,SIMC_Ave,label='SIMC',linestyle='dotted')
plt.plot(Manual_Time,Manual_Ave,label='Manual',linestyle='dashed')
plt.grid(which='both')
plt.ylabel('Displacement [cm]',fontsize = 15)
plt.xlabel('Time [s]',fontsize = 15)
plt.title('Average Responses of System Using Marble',fontsize = 15)
plt.axhline(15,label='Setpoint',linestyle='dotted',color = 'red')
plt.legend()